In [ ]:
#!pip install modin[ray]
#!pip install -U ipywidgets

In [19]:
import requests
from xml.etree import ElementTree as ET
import modin.pandas as pd
import os
import concurrent.futures

In [20]:
os.makedirs("datos", exist_ok=True)

In [21]:
endpoint = 'https://storage.googleapis.com/validaciones_tmsa/'
def descargar_archivo(carpeta_destino, destino, url = endpoint):
    file = os.path.join(carpeta_destino, destino.split('/')[-1])
    response = requests.get(url+destino)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
        print(f'Archivo descargado a {file}.')
    else:
        print(f'Error al descargar el archivo: {response.status_code}')

def descargar_archivos_paralelo(carpeta_destino, lista_destinos, url = endpoint):
    print(f'Descargando {len(lista_destinos)} archivos en paralelo...')
    with concurrent.futures.ThreadPoolExecutor() as executor:
        print(f'Usando {executor._max_workers} hilos...')
        executor.map(lambda x: descargar_archivo(carpeta_destino, x, url), lista_destinos)

In [22]:
def procesar_df(df):
    columns = ['Acceso_Estacion', 'Dispositivo', 'Estacion_Parada', 'Fecha_Transaccion']
    columns_upper = map(str.upper, columns)

    df = df[columns]
    df.columns = columns_upper

    df = df[df['ESTACION_PARADA'].str.contains('U. NACIONAL', na=False)].drop('ESTACION_PARADA', axis=1).convert_dtypes()
    df['DISPOSITIVO'] = df['DISPOSITIVO'].astype('int32')
    df['FECHA_TRANSACCION'] = pd.to_datetime(df['FECHA_TRANSACCION'], format='%Y-%m-%d %H:%M:%S')

    df.sort_values(by=['FECHA_TRANSACCION'], ascending=[True], inplace=True)

    return df

In [23]:
def procesar_df_lista(carpeta, file):
    f_name = os.path.join(carpeta, file.split('/')[1])

    try:
        df_tmp = pd.read_csv(f_name)
        df_tmp = procesar_df(df_tmp)
        print(f'Data frame {f_name} procesado')
        try:
            os.remove(f_name)
            print(f'Archivo {f_name} eliminado')
        except Exception as e:
            print(f'Error eliminando el archivo {f_name}: {e}')
            return None
    except Exception as e:
        print(f'Error procesando el df {f_name}: {e}')
        return None
    return df_tmp

In [24]:
response = requests.get(endpoint)
xml_data = response.text

root = ET.fromstring(xml_data)

In [25]:
files = []

namespace = {'ns': 'http://doc.s3.amazonaws.com/2006-03-01'}
total_size = 0
for element, size in zip(root.iterfind('.//ns:Key', namespace), root.iterfind('.//ns:Size', namespace)):
    if 'validacionTroncal202401' in element.text or 'validacionTroncal202402' in element.text or 'validacionTroncal202403' in element.text:
        files.append(element.text)
        total_size += int(size.text)

In [26]:
total_size_gb = total_size / (1024**3)
print(f'Tamaño total: {total_size_gb:.2f} GB')

Tamaño total: 29.39 GB


In [27]:
len(files)

57

In [28]:
import shutil

total, used, free = shutil.disk_usage("/")
free_gb = free / (1024**3)

print(f"Espacio libre en el disco duro: {free_gb:.2f} GB")


Espacio libre en el disco duro: 14.61 GB


In [29]:
ingresos_primer_trimestre_2024 = pd.DataFrame()

length = len(files)
batches = 6
for i in range(batches):
    print(f'Procesando batch {i+1}/{batches}...')
    files_batch = files[i*length//batches:(i+1)*length//batches]
    descargar_archivos_paralelo("datos", files_batch)
    dataframes = [procesar_df_lista("datos", f) for f in files_batch]
    ingresos_primer_trimestre_2024 = pd.concat([ingresos_primer_trimestre_2024] + dataframes, join='outer', ignore_index=True)

print('**********TODOS LOS ARCHIVOS HAN SIDO PROCESADOS**********')

Descargando 9 archivos en paralelo...
Usando 8 hilos...
Archivo descargado a datos/validacionTroncal20240204.csv.
Archivo descargado a datos/validacionTroncal20240211.csv.
Archivo descargado a datos/validacionTroncal20240210.csv.
Archivo descargado a datos/validacionTroncal20240130.csv.
Archivo descargado a datos/validacionTroncal20240131.csv.
Archivo descargado a datos/validacionTroncal20240205.csv.
Archivo descargado a datos/validacionTroncal20240202.csv.
Archivo descargado a datos/validacionTroncal20240201.csv.
Archivo descargado a datos/validacionTroncal20240207.csv.
Data frame datos/validacionTroncal20240130.csv procesado
Archivo datos/validacionTroncal20240130.csv eliminado
Data frame datos/validacionTroncal20240131.csv procesado
Archivo datos/validacionTroncal20240131.csv eliminado
Data frame datos/validacionTroncal20240201.csv procesado
Archivo datos/validacionTroncal20240201.csv eliminado
Data frame datos/validacionTroncal20240202.csv procesado
Archivo datos/validacionTroncal2

In [30]:
ingresos_primer_trimestre_2024.to_csv('ingresos_primer_trimestre_2024.csv')

In [87]:
ingresos_primer_trimestre_2024 = pd.read_csv('ingresos_primer_trimestre_2024.csv')

In [88]:
ingresos_primer_trimestre_2024.head()

,Unnamed: 0,ACCESO_ESTACION,DISPOSITIVO,FECHA_TRANSACCION
0,0,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30 04:43:45
1,1,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30 04:44:13
2,2,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30 04:46:42
3,3,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30 04:50:28
4,4,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000546,2024-01-30 04:51:55


In [89]:
ingresos_primer_trimestre_2024.drop('Unnamed: 0', axis=1, inplace=True)

In [90]:
ingresos_primer_trimestre_2024.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 581523 entries, 0 to 581522
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ACCESO_ESTACION    581523 non-null  object
 1   DISPOSITIVO        581523 non-null  int64 
 2   FECHA_TRANSACCION  581523 non-null  object
dtypes: int64(1), object(2)
memory usage: 13.3+ MB


In [91]:
ingresos_primer_trimestre_2024['FECHA_TRANSACCION'] = pd.to_datetime(ingresos_primer_trimestre_2024['FECHA_TRANSACCION'])
ingresos_primer_trimestre_2024['ACCESO_ESTACION'] = ingresos_primer_trimestre_2024['ACCESO_ESTACION'].astype('string')

In [92]:
ingresos_primer_trimestre_2024['HORA'] = ingresos_primer_trimestre_2024['FECHA_TRANSACCION'].dt.strftime('%H:%M:%S')
ingresos_primer_trimestre_2024['DIA'] = ingresos_primer_trimestre_2024['FECHA_TRANSACCION'].dt.weekday
ingresos_primer_trimestre_2024['FECHA_TRANSACCION'] = ingresos_primer_trimestre_2024['FECHA_TRANSACCION'].dt.date

In [93]:
ingresos_primer_trimestre_2024.head()

,ACCESO_ESTACION,DISPOSITIVO,FECHA_TRANSACCION,HORA,DIA
0,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30,04:43:45,1
1,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30,04:44:13,1
2,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30,04:46:42,1
3,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30,04:50:28,1
4,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000546,2024-01-30,04:51:55,1


In [95]:
ingresos_primer_trimestre_2024.to_csv('ingresos_primer_trimestre_2024.csv')

In [1]:
import pandas as pd

In [2]:
ingresos_primer_trimestre_2024 = pd.read_csv('ingresos_primer_trimestre_2024.csv')

In [3]:
ingresos_primer_trimestre_2024.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
ingresos_primer_trimestre_2024.head()

,ACCESO_ESTACION,DISPOSITIVO,FECHA_TRANSACCION,HORA,DIA
0,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30,04:43:45,1
1,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30,04:44:13,1
2,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30,04:46:42,1
3,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000547,2024-01-30,04:50:28,1
4,(03) BATERIA UNO VAGON SUR U. NACIONAL,10000546,2024-01-30,04:51:55,1


In [4]:
ingresos_primer_trimestre_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581523 entries, 0 to 581522
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ACCESO_ESTACION    581523 non-null  object
 1   DISPOSITIVO        581523 non-null  int64 
 2   FECHA_TRANSACCION  581523 non-null  object
 3   HORA               581523 non-null  object
 4   DIA                581523 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 22.2+ MB


In [6]:
ingresos_primer_trimestre_2024['FECHA_TRANSACCION'] = pd.to_datetime(ingresos_primer_trimestre_2024['FECHA_TRANSACCION'])
ingresos_primer_trimestre_2024['ACCESO_ESTACION'] = ingresos_primer_trimestre_2024['ACCESO_ESTACION'].astype('string')
ingresos_primer_trimestre_2024['HORA'] = ingresos_primer_trimestre_2024['FECHA_TRANSACCION'].dt.strftime('%H:%M:%S')

In [7]:
ingresos_primer_trimestre_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581523 entries, 0 to 581522
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ACCESO_ESTACION    581523 non-null  string        
 1   DISPOSITIVO        581523 non-null  int64         
 2   FECHA_TRANSACCION  581523 non-null  datetime64[ns]
 3   HORA               581523 non-null  object        
 4   DIA                581523 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1), string(1)
memory usage: 22.2+ MB


In [9]:
ingresos_primer_trimestre_2024['FECHA_TRANSACCION'].unique()

<DatetimeArray>
['2024-01-30 00:00:00', '2024-01-31 00:00:00', '2024-02-01 00:00:00',
 '2024-02-02 00:00:00', '2024-02-04 00:00:00', '2024-02-05 00:00:00',
 '2024-02-07 00:00:00', '2024-02-10 00:00:00', '2024-02-11 00:00:00',
 '2024-02-12 00:00:00', '2024-02-13 00:00:00', '2024-02-14 00:00:00',
 '2024-02-15 00:00:00', '2024-02-16 00:00:00', '2024-02-17 00:00:00',
 '2024-02-18 00:00:00', '2024-02-19 00:00:00', '2024-02-20 00:00:00',
 '2024-02-21 00:00:00', '2024-02-22 00:00:00', '2024-02-23 00:00:00',
 '2024-02-24 00:00:00', '2024-02-25 00:00:00', '2024-02-27 00:00:00',
 '2024-02-28 00:00:00', '2024-02-29 00:00:00', '2024-03-01 00:00:00',
 '2024-03-02 00:00:00', '2024-03-03 00:00:00', '2024-03-04 00:00:00',
 '2024-03-05 00:00:00', '2024-03-06 00:00:00', '2024-03-07 00:00:00',
 '2024-03-08 00:00:00', '2024-03-09 00:00:00', '2024-03-10 00:00:00',
 '2024-03-11 00:00:00', '2024-03-12 00:00:00', '2024-03-13 00:00:00',
 '2024-03-14 00:00:00', '2024-03-15 00:00:00', '2024-03-16 00:00:00',
 '20